### We have cleaned the data and stored in appropriate folders and now we move on to modelling component.

## Load Libraries

In [12]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.stats import mode
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

## Clean and Load Dataset

In [19]:
# Load dataset
def load_data(test_size=0.4):
    x, y = [], []
    for file in os.listdir(os.path.join('data', 'Parsed_Capuchinbird_Clips')):
        # Load audio file
        signal, sr = librosa.load(os.path.join('data', 'Parsed_Capuchinbird_Clips', file), sr=None, duration=5)
        # Extract features
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=30)
        chroma = librosa.feature.chroma_stft(y=signal, sr=sr)
        contrast = librosa.feature.spectral_contrast(y=signal, sr=sr,n_bands=1)
        mel_spectrogram = librosa.feature.melspectrogram(y=signal, sr=sr)
        spectral_centroids = librosa.feature.spectral_centroid(y=signal, sr=sr)
        spectral_rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sr)
        features = np.concatenate([mfccs.mean(axis=1), chroma.mean(axis=1), contrast.mean(axis=1), spectral_centroids.mean(axis=1),
                               spectral_rolloff.mean(axis=1),
                               mel_spectrogram.mean(axis=1)])
        # Append features and labels
        x.append(features)
        y.append(1)
    for file in os.listdir(os.path.join('data', 'Parsed_Not_Capuchinbird_Clips')):
        # Load audio file
        signal, sr = librosa.load(os.path.join('data', 'Parsed_Not_Capuchinbird_Clips', file), sr=None, duration=5)
        # Extract features
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=30)
        chroma = librosa.feature.chroma_stft(y=signal, sr=sr)
        contrast = librosa.feature.spectral_contrast(y=signal, sr=sr,n_bands=1)
        mel_spectrogram = librosa.feature.melspectrogram(y=signal, sr=sr)
        spectral_centroids = librosa.feature.spectral_centroid(y=signal, sr=sr)
        spectral_rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sr)
        features = np.concatenate([mfccs.mean(axis=1), chroma.mean(axis=1), contrast.mean(axis=1), spectral_centroids.mean(axis=1),
                               spectral_rolloff.mean(axis=1),
                               mel_spectrogram.mean(axis=1)])
        # Append features and labels
        x.append(features)
        y.append(0)
    # Convert to numpy arrays
    x, y = np.array(x), np.array(y)
    # Encode labels
    encoder = LabelEncoder()
    y = encoder.fit_transform(y)
    # Split into train and test sets - with class balancing using stratify technique
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size,stratify=y, random_state=42)
    # Scale features
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    return x_train, x_test, y_train, y_test

## Train and Evaluate Models

In [20]:
# Train and evaluate models
#Grid Search 
param_grid = {'n_estimators': [50, 100, 200, 500],
              'max_depth': [10, 20, 30, None]}

def train_evaluate_models(x_train, x_test, y_train, y_test):
    models = [RandomForestClassifier(n_estimators=100, random_state=42),
              SVC(kernel="rbf", gamma="scale", C=1.0),
              KNeighborsClassifier(n_neighbors=5)]
    for model in models:
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        acc = accuracy_score(y_test, y_pred)
        print(type(model).__name__, "accuracy:", acc)
        print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

x_train, x_test, y_train, y_test = load_data()
train_evaluate_models(x_train, x_test, y_train, y_test)

/home/himanshu/.local/lib/python3.10/site-packages/librosa/core/pitch.py:102: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


RandomForestClassifier accuracy: 0.9907407407407407
Confusion matrix:
 [[235   2]
 [  1  86]]
SVC accuracy: 0.9783950617283951
Confusion matrix:
 [[236   1]
 [  6  81]]
KNeighborsClassifier accuracy: 0.9753086419753086
Confusion matrix:
 [[235   2]
 [  6  81]]


In [23]:
# Train Bagging Classifier
bagging_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                 n_estimators=50,
                                 max_samples=0.5,
                                 max_features=0.5)
bagging_clf.fit(x_train, y_train)
y_pred_bagging = bagging_clf.predict(x_test)
print("Bagging classifier accuracy:", accuracy_score(y_test, y_pred_bagging))
print(classification_report(y_test, y_pred_bagging, target_names=["0","1"]))

# Train an AdaBoost classifier
adaboost_clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                                  n_estimators=200,
                                  learning_rate=0.1)
adaboost_clf.fit(x_train, y_train)
y_pred_adaboost = adaboost_clf.predict(x_test)
print("AdaBoost classifier accuracy:", accuracy_score(y_test, y_pred_adaboost))
print(classification_report(y_test, y_pred_adaboost, target_names=["0","1"]))

# Combine the predictions of the Bagging and AdaBoost classifiers using majority voting
y_pred_ensemble = np.array([mode([y_pred_bagging[i], y_pred_adaboost[i]])[0][0] for i in range(len(y_pred_bagging))])
print("Ensemble accuracy:", accuracy_score(y_test, y_pred_ensemble))
print(classification_report(y_test, y_pred_ensemble, target_names=["0","1"]))

/home/himanshu/.local/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging classifier accuracy: 0.9938271604938271
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       237
           1       0.99      0.99      0.99        87

    accuracy                           0.99       324
   macro avg       0.99      0.99      0.99       324
weighted avg       0.99      0.99      0.99       324



/home/himanshu/.local/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoost classifier accuracy: 0.9783950617283951
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       237
           1       0.98      0.94      0.96        87

    accuracy                           0.98       324
   macro avg       0.98      0.97      0.97       324
weighted avg       0.98      0.98      0.98       324

Ensemble accuracy: 0.9814814814814815
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       237
           1       0.99      0.94      0.96        87

    accuracy                           0.98       324
   macro avg       0.98      0.97      0.98       324
weighted avg       0.98      0.98      0.98       324



/tmp/ipykernel_344403/2923319603.py:20: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred_ensemble = np.array([mode([y_pred_bagging[i], y_pred_adaboost[i]])[0][0] for i in range(len(y_pred_bagging))])


## Conclusion

#### We can see the models are giving highly accurate results on cleaned data with very limited False Positives and False Negatives in the Confusion Matrix